In [1]:
from modeling.gen6.api import symbolic, acos
from modeling.gen7.api7 import MFunctionalSet, MFunctionalSetLeaf, FunctionSympy
from engine.torchdata import load_vals, print_formatted_table
import numpy as np

In [2]:
h, a, T, g, d, r = symbolic('h', 'a', 'T', 'g', 'd', 'r')
R, mu = 6371e3, 3.986e14
Orbit = MFunctionalSetLeaf(
    a==h+R, 
    T==2*np.pi*(a**3/mu)**0.5, 
    g==1/np.pi*acos(R/a), 
    d==g+0.5, 
    r==(h**2+2*R*h)**0.5
) # by default configured to elimination of state variables

m_A, P_c, P_T, E_b, m_b = symbolic('m_A', 'P_c', 'P_T', 'E_b', 'm_b')
eta_A, rho_A, rho_b, P_l, A, Q = 0.3, 10, 0.002, 12, 0.05, 1365
Power = MFunctionalSetLeaf(
    m_A==rho_A*A, 
    P_c==d*A*Q*eta_A, 
    P_T==P_c-P_l, 
    E_b==P_c*T/d, 
    m_b==rho_b*E_b
)

X_r, D_p, D, m_p = symbolic('X_r', 'D_p', 'D', 'm_p')
rho_p, l_v, B, N = 2, 500, 8, 2000
Payload = MFunctionalSetLeaf(
    D_p==1.22*l_v*h/X_r, 
    D==2*np.pi*R*B*N/X_r, 
    m_p==rho_p*D_p**1.5
)

MDA = MFunctionalSet(Orbit, Power, Payload)
FPF = MDA.subsetof(X_r <= 5).minimize(m_p+m_b)

In [5]:
_,_,_,_, indices = Power.gather_sets()
f = Power.build()
x0 = {"d": 0.6, "T": 6000}
xidxed = load_vals(x0, indices, isdict=True)
f.analysis(xidxed)
idxrev = {var.item():key for key,var in indices.items()}
print_formatted_table([f.analysis(xidxed)], indices, idxrev)

m_b   P_T   d   m_A E_b    P_c    T   
245.7 0.285 0.6 0.5 1.23e5 12.285 6000


In [12]:
_,_,_,_, indices = Orbit.gather_sets()
f = Orbit.build()
x0 = {"h": 400e3}
xidxed = load_vals(x0, indices, isdict=True)
f.analysis(xidxed)
idxrev = {var.item():key for key,var in indices.items()}
print_formatted_table([f.analysis(xidxed)], indices, idxrev)

g    d    r      h      a      T       
0.11 0.61 2.29e6 4.00e5 6.77e6 5544.858


In [ ]:
SPF_IDF = FPF.config(parallel=[Orbit, Power, Payload])
SPF_AAO = FPF.config(residuals=Orbit.supersets+Power.supersets+Payload.supersets)
SPF_MDF1 = FPF.config(elim=[Orbit, Power, Payload])
SPF_MDF2 = FPF.config(elim=[MDA.config(parallel=[Orbit, Power, Payload])])